In [55]:
# Generate a dataset from Gaussian Mixture Model

In [56]:
import numpy as np
from scipy import stats
import random

In [57]:
# Method for generating a dataset from a gaussian mixture model
def genMixtureModelDataset(numPoints, numDims, numClusters):
    # Randomly initialize gaussians (define by mean)
    clusterCenters = np.random.rand(numClusters, numDims)
    # Generate data-points 
    # randomly and uniformly sample points from the gaussians defined by the cluster centers
    data = np.array([stats.multivariate_normal.rvs(clusterCenters[np.random.randint(0, numClusters)]) for i in range(0,numPoints)])
    targets = np.array([[-1 if random.random() < 0.5 else 1] for i in range(0, numPoints)])
    return data, targets

In [58]:
X, y = genMixtureModelDataset(10, 2, 3) # Generate dataset 
# Parameters: number of points, number of dimensions, number of mixture components

In [ ]:
# TODO: Learn K-NN Model, 